In [1]:
import os
import json
import glob
import h5py
import numpy as np
from tqdm.notebook import tqdm

import sys
sys.path.append('../../py/')

from cosmoSim import cosmoSim

In [2]:
outdir = '../../data_prods/'
data_dir = '/home/ryan/Data'
subdir_list = [os.path.join(data_dir, o) for o in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir,o))]

print(subdir_list)

['/home/ryan/Data/run_2cDM_L3N256_DM_powerm2m2_sigma0.1', '/home/ryan/Data/run_2cDM_L3N512_HY_power00_sigma1', '/home/ryan/Data/run_2cDM_L3N256_HY_power00_sigma0.1', '/home/ryan/Data/run_SIDM_L3N256_DM_power0_sigma1', '/home/ryan/Data/lost+found', '/home/ryan/Data/run_2cDM_L3N256_DM_power00_sigma1_dir_3', '/home/ryan/Data/run_2cDM_L3N256_HY_powerm2m2_sigma10', '/home/ryan/Data/run_CDM_L3N256_DM_dir_7', '/home/ryan/Data/run_2cDM_L3N256_DM_power00_sigma1_dir_8', '/home/ryan/Data/run_CDM_L5N256_HY', '/home/ryan/Data/run_2cDM_L3N128_HY_power00_sigma0.1_10e-7', '/home/ryan/Data/run_2cDM_L3N256_DM_power00_sigma0.1', '/home/ryan/Data/run_2cDM_L3N128_HY_power00_sigma0.1_var4', '/home/ryan/Data/run_2cDM_L3N256_DM_power00_sigma1_dir_4', '/home/ryan/Data/run_CDM_710_HY', '/home/ryan/Data/run_2cDM_710_HY_powerm2m2_sigma1', '/home/ryan/Data/run_2cDM_L3N256_DM_power00_sigma1_dir_1', '/home/ryan/Data/run_CDM_L3N128_HY', '/home/ryan/Data/run_2cDM_L3N256_DM_power00_sigma1_dir_6', '/home/ryan/Data/run_2

In [3]:
def get_masses(sbh_pos, rbins, coord, mass_table):

    r = (coord[:,0] - sbh_pos[0])**2 + (coord[:,1] - sbh_pos[1])**2 + (coord[:,2] - sbh_pos[2])**2

    m_in = []
    for R in rbins:
        m_in.append(mass_table[r <= R].sum())
    return np.array(m_in)
    

In [4]:
rewrite = False
max_no_subhalos = 25

for subdir in tqdm(subdir_list):
    run_name = subdir.split('/')[-1]

    #reject folders that aren't runs
    if 'run' not in run_name:
        continue
    
    fpath = os.path.join(outdir, run_name)
    try:
        os.mkdir(fpath)
    except:
        print(f'{fpath} already exists!')

    run = cosmoSim(run_name)
    force_scale = 2.8 * run.softening_length

    snaps = sorted(glob.glob(subdir + '/snap*'))
    fofs = sorted(glob.glob(subdir + '/fof*'))

    print(f'Calculating mass densities for run {run_name}...')

    for i, (snap, fof) in enumerate(zip(snaps, fofs)):
        
        fout = os.path.join(fpath, f'subhalo_densities_{i}')
        try:
            os.mkdir(fout)
        except:
            print(f'{fout} already exists!')
        # get density for each subhalo
        f = h5py.File(snap, "r")
        s = h5py.File(fof, "r")

        try:
            half_mass_radii = s["Subhalo/SubhaloHalfmassRad"][()]
        except:
            print(f'No subhalos formed for snapshot {i}...')
            continue
        
        # TODO: bodge for now, find better criteria
        if len(half_mass_radii) > max_no_subhalos:
            # just getting top 100
            large_enough = half_mass_radii > np.sort(half_mass_radii)[-(max_no_subhalos + 1)]
        else:
            large_enough = half_mass_radii > force_scale

        if large_enough.sum() == 0:
            print(f'Not enough large subhalos for snapshot {i}...')
            continue

        half_mass_radii = half_mass_radii[large_enough]
        sbh_positions = s["Subhalo/SubhaloPos"][()][large_enough]

        if not rewrite:
            if (len( glob.glob( os.path.join(fout, 'subhalo*.txt') ) ) == len(half_mass_radii)):
                print(f'Mass density files exist for {run_name}.')
                print('Skipping...')
                continue
        
        print(f'Calculating {len(half_mass_radii)} subhalo density profiles for snapshot {i}...')

        dm_pos = f.get("PartType1/Coordinates")[()]
        dm_number = f["Header"].attrs["NumPart_Total"][1]
        dm_mass = f["Header"].attrs["MassTable"][1] * np.ones(dm_number)

        if run.baryon_type == 'HY':
            by_pos = f.get("PartType0/Coordinates")[()]
            by_mass = f.get("PartType0/Masses")[()]

        for j, (sbh_pos, hmr) in enumerate(zip(sbh_positions, half_mass_radii)):
            print(f'Calculating subhalo density profile for subhalo {j}...')

            rbins = np.linspace(force_scale/10, 2*hmr, num=1000)
            volumes = 4/3 * np.pi * rbins**3

            coord = dm_pos
            mass_table = dm_mass

            masses = get_masses(sbh_pos, rbins, coord, mass_table)

            if run.baryon_type == 'HY':
                coord = by_pos
                mass_table = by_mass

                masses += get_masses(sbh_pos, rbins, coord, mass_table)
            
            densities = masses / volumes

            fname = os.path.join(fout, f'subhalo_{j}.txt')

            np.savetxt(fname, (rbins, densities))

            print(f'Saved subhalo {j} density profile at {fname}')




  0%|          | 0/74 [00:00<?, ?it/s]

../../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma0.1 already exists!
Calculating mass densities for run run_2cDM_L3N256_DM_powerm2m2_sigma0.1...
../../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma0.1/subhalo_densities_0 already exists!
No subhalos formed for snapshot 0...
../../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma0.1/subhalo_densities_1 already exists!
Calculating 25 subhalo density profiles for snapshot 1...
Calculating subhalo density profile for subhalo 0...
Saved subhalo 0 density profile at ../../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma0.1/subhalo_densities_1/subhalo_0.txt
Calculating subhalo density profile for subhalo 1...
Saved subhalo 1 density profile at ../../data_prods/run_2cDM_L3N256_DM_powerm2m2_sigma0.1/subhalo_densities_1/subhalo_1.txt
Calculating subhalo density profile for subhalo 2...


KeyboardInterrupt: 